<a href="https://colab.research.google.com/github/JanXanevo/PrototypeRecommendations/blob/main/03_gardenOnlineAprioriPNK_14_11_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
# Gartenprodukte aus online laden
df = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/onlineGardenProducts.csv')

In [ ]:
df = df.sort_values('hybris_bestellnummer')

In [ ]:
# Bestellungen entfernen, welche nur 1 Produkt enthalten

df_filtered = df [df['hybris_bestellnummer'].duplicated(keep=False)]

In [ ]:
# gruppieren um zu jeder Bestellung eine List von Produkten (hier PNKs) zu erhalten

grouped_data = df_filtered.groupby('hybris_bestellnummer')['PRODUCT_NAME_CATEGORY'].apply(list).reset_index()

# für Nummern die PNK durch 'produktnr' ersetzen

In [ ]:
grouped_data['PRODUCT_NAME_CATEGORY'] = grouped_data['PRODUCT_NAME_CATEGORY'].apply(lambda x: sorted(x))


In [ ]:
# TransactionEncoder wandelnt die Daten in ein frame aus true false um

from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()

encoded_data = encoder.fit(grouped_data['PRODUCT_NAME_CATEGORY']).transform(grouped_data['PRODUCT_NAME_CATEGORY'])

In [ ]:
df_encoded = pd.DataFrame(encoded_data, columns = encoder.columns_)

In [ ]:
# apriori algorithmus

frequent_itemsets = apriori(df_encoded, min_support=0.004, use_colnames=True)
# der min-support bestimmt wie viel support zugegeben wird und ab wann Folgerungen gelistet werden (0,004 --> 0,4% von 323252 = 1293)

# Extrahieren der Assoziationsregeln
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)


# zu niedriger treshhold führt im Graphen allerdings zu Verbindung zu vieler Produkte
# alternative fp-growth

# Anzeigen der Regeln
print(rules)

In [ ]:
df_rules = pd.DataFrame(rules)
df_rules = df_rules[['antecedents','consequents']]

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

df_exploded = df_rules.explode('antecedents').explode('consequents')

# Erstelle einen gerichteten Graphen
G = nx.from_pandas_edgelist(df_exploded, 'antecedents', 'consequents', create_using=nx.Graph())

# Durchlaufe den Graphen und erstelle für jede verbundene Komponente eine Zeile im DataFrame
components = [set(component) for component in nx.connected_components(G)]
df_projects = pd.DataFrame({'Projekt': [f'Projekt{i+1}' for i in range(len(components))],
                          'VerbundeneKnoten': components})

# Zeige das resultierende DataFrame an
print(df_projects)

In [ ]:
# Graph visuell anzeigen

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=1000, font_size=5.5)

In [ ]:
# Daten entgruppieren für KG input

new_data = []

for index, row in df_projects.iterrows():
    project = row['Projekt']
    connected_nodes = row['VerbundeneKnoten']

    for node in connected_nodes:
        new_data.append({'Projekt': project, 'VerbundenerKnoten': node})

# Neues DataFrame erstellen
projects_ungrouped = pd.DataFrame(new_data)


In [ ]:
projects_path ='/content/drive/My Drive/BAHAG/Data/projectsGardenOnline.csv'
df_projects.to_csv(projects_path, index=False)

# projects_ungrouped_path = '/content/drive/My Drive/BAHAG/Data/projectsUngroupedGardenOnline.csv'
# projects_ungrouped.to_csv(projects_ungrouped_path, index=False)